In [16]:
from dask.distributed import Client

client = await Client(processes=False, asynchronous=True, n_workers=4, threads_per_worker=2)

/Users/matthewiannucci/Developer/gribberish/python/examples/env/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 55176 instead
  warnings.warn(


In [8]:
import xarray as xr
import fsspec

# open dataset as zarr object using fsspec reference file system and xarray
# fs = fsspec.filesystem("filecache", 
#                        expiry_time=4,
#                        target_protocol='reference', 
#                        target_options={
#                            'fo': f'./hrrr_hourly/hrrr_kerchunk.json', 
#                            'remote_protocol': 's3', 
#                            'remote_options': {'anon':True, 'use_ssl': False}
#                        })
fs = fsspec.filesystem("reference", fo=f'./hrrr_hourly/hrrr_kerchunk.json', remote_protocol='s3', remote_options={'anon':True, 'use_ssl': False})
m = fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr", backend_kwargs=dict(consolidated=False), chunks={'time': 1})
ds

<xarray.Dataset>
Dimensions:    (time: 48, y: 1059, x: 1799)
Coordinates:
    latitude   (y, x) float64 dask.array<chunksize=(1059, 1799), meta=np.ndarray>
    longitude  (y, x) float64 dask.array<chunksize=(1059, 1799), meta=np.ndarray>
  * time       (time) datetime64[s] 2023-07-22T19:00:00 ... 2023-07-24T18:00:00
  * x          (x) float64 -2.701e+06 -2.698e+06 ... 2.69e+06 2.693e+06
  * y          (y) float64 -1.581e+06 -1.578e+06 ... 1.59e+06 1.593e+06
Data variables:
    apcp       (time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    cpofp      (time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    dpt        (time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    prate      (time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    pres       (time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    tmp        (time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    ugrd       (time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    vgrd       (time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    vis        (time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
Attributes:
    meta:     Generated with gribberishpy

In [9]:
import pyproj
to_xy = pyproj.Transformer.from_crs('epsg:4326', ds.prate.crs, always_xy=True).transform

In [10]:
import numpy as np

ROTCON_P = 0.622515
LON_XX_P = -97.5
LAT_TAN_P = 38.5

def wind_speed_dir(u, v, lng):
    """Calculate wind speed (in mph) and direction from u and v components.
    """
    angle2 = ROTCON_P*(lng-LON_XX_P)*0.017453
    sinx2 = np.sin(angle2)
    cosx2 = np.cos(angle2)

    un = cosx2 * u + sinx2 * v
    vn = -sinx2 * u + cosx2 * v

    wind_speed = np.sqrt(un**2 + vn**2) * 2.23694
    wind_dir = (270 - np.arctan2(vn, un) * 180 / np.pi) % 360

    return wind_speed, wind_dir

In [11]:
async def get_forecast(lat, lng):
    x_sel, y_sel = to_xy(lng, lat)
    
    selected_data = ds.interp(x=x_sel, y=y_sel, method='linear')
    time = selected_data.time
    
    temp_task = client.compute(selected_data.tmp)
    u_task = client.compute(selected_data.ugrd)
    v_task = client.compute(selected_data.vgrd)
    apcp_task = client.compute(selected_data.apcp)

    # Fetch each variable asynchronously
    temp, u, v, apcp = await client.gather([temp_task, u_task, v_task, apcp_task])

    wind_speed, wind_dir = wind_speed_dir(u.values, v.values, lng)

    return {
        'time': time.values,
        'temp': temp.values,
        'wind_speed': wind_speed,
        'wind_dir': wind_dir,
        'apcp': apcp.values
    }

In [17]:
forecast = await get_forecast(41.42717976016072, -71.462122760827)
forecast

{'time': array(['2023-07-22T19:00:00', '2023-07-22T20:00:00',
        '2023-07-22T21:00:00', '2023-07-22T22:00:00',
        '2023-07-22T23:00:00', '2023-07-23T00:00:00',
        '2023-07-23T01:00:00', '2023-07-23T02:00:00',
        '2023-07-23T03:00:00', '2023-07-23T04:00:00',
        '2023-07-23T05:00:00', '2023-07-23T06:00:00',
        '2023-07-23T07:00:00', '2023-07-23T08:00:00',
        '2023-07-23T09:00:00', '2023-07-23T10:00:00',
        '2023-07-23T11:00:00', '2023-07-23T12:00:00',
        '2023-07-23T13:00:00', '2023-07-23T14:00:00',
        '2023-07-23T15:00:00', '2023-07-23T16:00:00',
        '2023-07-23T17:00:00', '2023-07-23T18:00:00',
        '2023-07-23T19:00:00', '2023-07-23T20:00:00',
        '2023-07-23T21:00:00', '2023-07-23T22:00:00',
        '2023-07-23T23:00:00', '2023-07-24T00:00:00',
        '2023-07-24T01:00:00', '2023-07-24T02:00:00',
        '2023-07-24T03:00:00', '2023-07-24T04:00:00',
        '2023-07-24T05:00:00', '2023-07-24T06:00:00',
        '2023-07-24T

In [23]:
z_ds = ds.assign_coords(time=ds.time.astype("datetime64[ns]"))
z_ds = client.compute(z_ds)
z_ds.to_zarr("./hrrr_hourly/hrrr.zarr")

type: Invalid value for attr 'proj_params': {'lat_0': 38.5, 'lat_1': 38.5, 'lat_2': 38.5, 'lon_0': 262.5, 'proj': 'lcc'}. For serialization to netCDF files, its value must be of one of the following types: str, Number, ndarray, number, list, tuple